In [19]:
#Ignorer les warnings

import warnings
warnings.filterwarnings("ignore")

In [20]:
#Importation des librairies nécessaires

import pandas as pd
import numpy as np
from datetime import datetime
import csv

# Importation des données

In [21]:
#Importation du fichier | MODIFIER LE NOM DU FICHIER A CHAQUE ITERATION (lien en rouge)
data = pd.read_excel(
    r"C:\Users\MOUA\OneDrive\Documents\Python Scripts\Vil\2021_07_18267_VILMORIN_JARDIN.xlsx")

In [22]:
#Lecture du fichiers ventes pour faire l'automatisation de l'index
ventes = pd.read_csv('table_vente.csv', sep = ";")

# MODE DE L'ALGO : A MODIFIER A CHAQUE UTILISATION

In [23]:
#Vérification de l'orthographe
Enseigne = ''

while Enseigne not in ['1','2','3','4','5'] :
    Enseigne = input("Nom de l'enseigne (sans MAJ ni é):")

#Mettre la date année/mois des sorties caisses
date = input("Date sous format AAAA/MM :")

Nom de l'enseigne (sans MAJ ni é):bricomarche
Date sous format AAAA/MM :2021/07


# Exploration des données

In [24]:
#Visualisation des données
data.head()

,Mois de référence,CNUF,Fournisseur,EAN13,Libelle Commercial,PDV,ville,Dept,Total Qté Vendue Année N-1,Qté Vendue Période N-1,Qté Vendue Période N,% Evolution Qté,Total CATTC Vendue Année N-1,CATTC Vendue Période N-1,CATTC Vendue Période N,% Evolution CATTC,Qté Stock,Détention,Référencement
0,2021/07,18267,VILMORIN JARDIN,3505390924358,GAZON REGARNI.ENROBE 1KG,20142,PLEINE FOUGERES,35,3.0,0.0,0.0,0.0,36.78,0.00,0.00,0.0,0.0,0,0
1,2021/07,18267,VILMORIN JARDIN,3182670224614,J F OISEAUX 7M2 65G,20231,PALAMINY,31,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,5.0,1,1
2,2021/07,18267,VILMORIN JARDIN,3211500008664,EPINARD VIROFLAY VL 15G,20219,Colleville Montgomery,14,2.0,1.0,1.0,0.0,4.50,2.25,2.25,0.0,14.0,1,1
3,2021/07,18267,VILMORIN JARDIN,3182670236303,FILM PAILL BIO 1.50X4M,3596,LAMORLAYE,60,2.0,0.0,0.0,0.0,12.60,0.00,0.00,0.0,15.0,1,1
4,2021/07,18267,VILMORIN JARDIN,3505390972137,7 TUTEURS BAMBOU 90CM,2348,MAURIAC,15,43.0,10.0,4.0,-1.5,66.22,15.40,6.16,-1.5,28.0,1,1


In [25]:
#Détection des valeurs manquantes
data.isnull().sum()

Mois de référence                  0
CNUF                               0
Fournisseur                        0
EAN13                              0
Libelle Commercial                 0
PDV                                0
ville                           1288
Dept                               0
Total Qté Vendue Année N-1         0
Qté Vendue Période N-1             0
Qté Vendue Période N               0
% Evolution Qté                    0
Total CATTC Vendue Année N-1       0
CATTC Vendue Période N-1           0
CATTC Vendue Période N             0
% Evolution CATTC                  0
Qté Stock                          0
Détention                          0
Référencement                      0
dtype: int64

In [26]:
ventes.tail()

,id,Gencode,Code_article,Volume,Valeur HT,Valeur TTC,PDV,date,Enseigne


In [27]:
if ventes.duplicated().sum() > 0 :
    raise SystemExit("DOUBLON DANS LES VENTES")

In [28]:
#Vérification du principe d'unicité du fichier des ventes
if ventes['id'].duplicated().sum() > 0 :
    raise SystemExit("Principe d'unicité non respecté : id ")

# Mise en forme et traitement des données 

In [29]:
tab = pd.DataFrame()

In [30]:
#Pré-traitement, suppression des lignes vides et des lignes inutiles

match Enseigne : 
    case "1" :
    
        #Supprime les lignes sans volumes ni stock | Gain de mémoire
        data.drop(data[(data['Qté Vendue Période N'] == 0)].index,inplace = True)

        #Supprime la ligne Totaux
        data = data.dropna(subset=['EAN13'])
    
    case "2" :
    
        #Supprime les lignes sans volumes ni stock | Gain de mémoire
        data.drop(data[(data[' QTE VENTE '] == 0)].index,inplace = True)

        #Supression de la ligne totaux à la fin
        data.drop(data[data['CODE ARTICLE '] == ' TOTAUX '].index,inplace = True)

        #Suppression de la dernière ligne vide mais avec des espaces à l'intérieur
        data.drop(data[data[' CODE ARTICLE FRN '] == '  '].index,inplace = True)
        data = data.dropna()
    
    case '3' :
        data = data.dropna(subset = ['Référence'])
    
    case '4' :
        data = data.dropna(subset = ['EAN Article'])

        #Si des milliers présent : espace insécable
        data['CA HT'] = data['CA HT'].astype(str)
        data['CA HT'] = data['CA HT'].str.replace('\u00A0', '')
        data['CA HT'] = data['CA HT'].str.replace(' ', '')
        data['CA HT'] = data['CA HT'].str.replace('€', '')
        data['CA HT'] = data['CA HT'].str.replace(',', '.')

        data['CA HT'] = data['CA HT'].astype(float)
    
    case '5' :
        data.columns = data.iloc[0]
        data.dropna(subset = ['CD_ARTICLE'], inplace = True)
        data.drop(data[(data.index == 0)].index,inplace = True)
    
    #Stop le programme si erreur
    case _ :
        raise SystemExit("Stop right there!")

In [31]:
#Création des nouvelles données pour la base de données en fonction de l'enseigne des données

#Itération pour l'enseigne 1

match Enseigne :
    case "1" :


        tab['Gencode'] = data['EAN13']
        tab['Code_article'] = np.nan
        tab['Volume'] = data['Qté Vendue Période N']
        tab['Valeur HT'] = np.nan
        tab['Valeur TTC'] = data['CATTC Vendue Période N']
        tab['PDV'] = data['PDV']
        tab['date'] = date
        tab['Enseigne'] = "1"

        m = tab['Volume'] > 0

        new = tab.groupby('Gencode').agg({
                                          'Code_article':'first',
                                          'Volume':'sum',
                                          'Valeur HT': 'sum',
                                          'Valeur TTC' : 'sum',
                                          'PDV' : lambda x: x.where(m).count() ,
                                          'date' : 'first',
                                          'Enseigne' : 'first'})

        new.reset_index(inplace= True)
        new.insert(0,'id', np.nan) 

        #Formate la colonne date en DATE et non en STRING
        new['date'] = pd.to_datetime(new['date'],format='%Y/%m')

    
    #Itération pour l'enseigne 2 
    case "2" :

        #Supprimer les deux dernières lignes : Le total + une ligne vide
        #data.drop(data.tail(2).index,inplace = True)

        tab['Gencode'] = data[' CODE EAN ']
        tab['Code_article'] = data['CODE ARTICLE ']
        tab['Volume'] = data[' QTE VENTE '].astype(float)
        tab['Valeur HT'] = data[' CA HT '].astype(float)
        tab['Valeur TTC'] = np.nan
        tab['PDV'] = np.nan
        tab['date'] = date
        tab['Enseigne'] = "2"

        new = tab.groupby('Gencode').agg({
                                          'Code_article':'first',
                                          'Volume':'sum',
                                          'Valeur HT': 'sum',
                                          'Valeur TTC' : 'sum',
                                          'PDV' : 'count' ,
                                          'date' : 'first',
                                          'Enseigne' : 'first'})

        new.reset_index(inplace= True)
        new.insert(0,'id', np.nan) 

       #Formate la colonne date en DATE et non en STRING
        tab['date'] = pd.to_datetime(tab['date'],format='%Y/%m')


    case '3' :
        tab['Gencode'] = data['ean'].astype(str)
        tab['Code_article'] = data['Référence'].astype(str)
        tab['Volume'] = data['Qté.']
        tab['Valeur HT'] = np.nan
        tab['Valeur TTC'] = np.nan
        tab['PDV'] = data['Code Mag']
        tab['date'] = date
        tab['Enseigne'] = "3"

        m = tab['Volume'] > 0

        new = tab.groupby('Gencode').agg({'Code_article':'first',
                                          'Volume':'sum',
                                          'Valeur HT': 'sum',
                                          'Valeur TTC' : 'sum',
                                          'PDV' : lambda x: x.where(m).count() ,
                                          'date' : 'first',
                                          'Enseigne' : 'first'})

        new.reset_index(inplace= True)
        new.insert(0,'id', np.nan) 

       #Formate la colonne date en DATE et non en STRING
        new['date'] = pd.to_datetime(tab['date'],format='%Y/%m')
        new['Gencode'] =  new['Gencode'].str.replace('.0','')
        new['Code_article'] =  new['Code_article'].str.replace('.0','')

    case '4' :

        tab['Gencode'] = data['EAN Article'].astype(str)
        tab['Code_article'] = np.nan
        tab['Volume'] = data['Quantité Vente']
        tab['Valeur HT'] = data['CA HT'].astype(float)
        tab['Valeur TTC'] = np.nan
        tab['PDV'] = data['GLN Magasin']
        tab['date'] = date
        tab['Enseigne'] = "4"

        m = tab['Volume'] > 0

        new = tab.groupby('Gencode').agg({
                                          'Code_article':'first',
                                          'Volume':'sum',
                                          'Valeur HT': 'sum',
                                          'Valeur TTC' : 'sum',
                                          'PDV' : lambda x: x.where(m).count() ,
                                          'date' : 'first',
                                          'Enseigne' : 'first'})

        new.reset_index(inplace= True)
        new.insert(0,'id', np.nan)
        new['Gencode'] =  new['Gencode'].str.replace('.0','')

        #Formate la colonne date en DATE et non en STRING
        new['date'] = pd.to_datetime(new['date'],format='%Y/%m')

    case '5' :
        tab['Gencode'] = data['CD_ARTICLE'].astype(str)
        tab['Code_article'] = np.nan
        tab['Volume'] = data['QTE'].astype(float)
        tab['Valeur HT'] = np.nan
        tab['Valeur TTC'] = data['CA_TTC'].astype(float)
        tab['PDV'] = np.nan
        tab['date'] = date
        tab['Enseigne'] = "5"

        m = tab['Volume'] > 0

        new = tab.groupby('Gencode').agg({
                                          'Code_article':'first',
                                          'Volume':'sum',
                                          'Valeur HT': 'sum',
                                          'Valeur TTC' : 'sum',
                                          'PDV' : lambda x: x.where(m).count() ,
                                          'date' : 'first',
                                          'Enseigne' : 'first'})

        new.reset_index(inplace= True)
        new.insert(0,'id', np.nan)
        new['Gencode'] =  new['Gencode'].str.replace('.0','')

        #Formate la colonne date en DATE et non en STRING
        new['date'] = pd.to_datetime(new['date'],format='%Y/%m')



    #Stop le programme si aucune enseigne répertorié est décrite
    case _ :
        raise SystemExit("Stop right there!")

# Vérification de l'intégrité des données

In [32]:
#Stop le système si erreurs et donc perte de données pour les volummes

a = round(new['Volume'].sum())

match Enseigne :
    case "1" :

        b = data['Qté Vendue Période N'].sum().round()

    case '2' :

        b = round(data[' QTE VENTE '].sum())
        
    case '3' :

        b = round(data['Qté.'].sum())

    case '4' :

        b = round(data['Quantité Vente'].sum())

    case '5' :
        b = round(data['QTE'].sum())

#Stop le programme si aucune enseigne répertorié est décrite
    case _ :
        raise SystemExit("Stop right there!")
    
#Vérification de l'intégrité des données des volumes
if a == b : print('Ok !')
        
else :
    print("Volume :",a,"Volume data brutes :",b)
    raise SystemExit("Volume différent !") 

Ok !


In [33]:
#Stop le système si erreurs et donc perte de données pour le CA

c = new['Valeur TTC'].sum().round()

match Enseigne :
    case "1" :

        d = data['CATTC Vendue Période N'].sum().round()


    case "2" :

        c = new['Valeur HT'].sum().round()
        d = round(data[' CA HT '].sum())


    case '3' :

        d = 0
        print('Pas de valeur, Ok !')

    
    case "4" :

        c = round(new['Valeur HT'].sum())
        d = round(data['CA HT'].sum())

    case '5' :
        d = round(data['CA_TTC'].sum())

            
#Stop le programme si aucune enseigne répertorié est décrite
    case _ :
        raise SystemExit("Stop right there!")

#Vérification de l'intégration des données pour le CA    
if c == d :
    print('Ok !')

else:
    print("Valeur :",c,"Valeur data brutes :",d)
        
    raise SystemExit("Valeurs différentes !")

Ok !


# Gestion de l'index

In [34]:
new['id'] = pd.RangeIndex(start = len(ventes), stop = len(new)+len(ventes),step = 1)

In [35]:
new.tail()

,id,Gencode,Code_article,Volume,Valeur HT,Valeur TTC,PDV,date,Enseigne
1536,1536,3585150006159,NaN,428.0,0.0,1096.81,160,2021-07-01,BCM
1537,1537,3999991666357,NaN,468.0,0.0,6645.36,145,2021-07-01,BCM
1538,1538,3999991666371,NaN,719.0,0.0,18513.88,244,2021-07-01,BCM
1539,1539,3999991698334,NaN,3.0,0.0,147.00,1,2021-07-01,BCM
1540,1540,7290012652080,NaN,11.0,0.0,223.72,10,2021-07-01,BCM


# Exportation des données en fichier CSV

In [36]:
# Exportation Rajoute sur un fichier existant (mode : append), sans le nom des colonnes (header)
#rajout des quote pour les str (gencode) pour éviter les effacements de formats
header = False
new.to_csv('table_vente.csv', index = False, mode = "a",header = header, sep = ';',decimal = ",",quoting=csv.QUOTE_NONNUMERIC)